In [8]:
import numpy as np
import numpy.random as rd
import pandas as pd
import pandas_datareader.data as web

connect google drive to this notebook so we can access/ save files there

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
symbols = pd.read_csv('/content/drive/My Drive/edav/SP500 Components.csv', index_col=False)

In [11]:
symbols = symbols[['Symbol']]

Following block reads sp500 tickers and puts them in a list

In [12]:
#symbols = pd.read_csv('/content/drive/My Drive/edav/sp500.csv', index_col=False)
symbols = list(symbols['Symbol'].values)
symbols = sorted(symbols)
symbols = ['BF-B' if x=='BF.B' else x for x in symbols]
symbols = ['BRK-B' if x=='BRK.B' else x for x in symbols]

In [13]:
print(symbols)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF-B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK-B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DL

We define starting and ending date 

In [14]:
start_date = pd.to_datetime('2018-08-1')
end_date   = pd.to_datetime('2020-08-1')

We define our data scraper from yahoo

In [15]:
def scrape_yahoo(stock_name, start_date, end_date):
  # scrape data of each stock from yahoo
  try:
    # here we pass in the ticker and the start and end data and that returns 
    # the data we need
    df = web.DataReader(stock_name,'yahoo', start_date, end_date)
    df = df[['Adj Close','Volume']]
    
    if stock_name =='BF-B':
      stock_name = 'BF.B'
    
    if stock_name =='BRK-B':
      stock_name = 'BRK.B'
    
    df['Symbol'] =  stock_name
    find_flag = 1
    return df, find_flag
  except KeyError:
    print("Could not find data on ".format(stock_name))
    find_flag = 0
    return pd.DataFrame(), find_flag

For all tickers in list, scrape data from yahoo and save it under "sp500_data" folder in our shared drive 

In [16]:
count = 0
table = dict()
for stock in symbols:
  count += 1
  if count % 100 == 0:
    print(f"{count} stocks out of {len(symbols)} completed")

  TS = scrape_yahoo(stock, start_date, end_date)[0]
  table[stock] = len(TS)
  TS.to_csv('/content/drive/My Drive/edav/sp500_data/'+stock+'.csv', header=TS.columns,)

100 stocks out of 505 completed
200 stocks out of 505 completed
300 stocks out of 505 completed
400 stocks out of 505 completed
Could not find data on 
500 stocks out of 505 completed


In [17]:
import glob

lists = glob.glob('/content/drive/My Drive/edav/sp500_data/*')

In [18]:
d = pd.read_csv(lists[0])
final_df = d

count = 0
for f in lists[1:]:
  d = pd.read_csv(f)
  final_df = final_df.append(d)

In [19]:
final_df.set_index(['Date', 'Symbol']).to_csv('/content/drive/My Drive/edav/data/sp500_data.csv')